In [37]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

Загрузим датасет из лабораторной 5.2

In [6]:
data = pd.read_csv('diabetes.csv')
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [9]:
X = data.drop(["Outcome"], axis=1)
y = data["Outcome"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

In [38]:
bst = XGBClassifier(learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8, random_state=121)
start_time = time.time()
bst.fit(X_train, y_train)
time_for_fit = time.time() - start_time
preds = bst.predict(X_test)
print("F1:", f1_score(y_test, preds))
print("Время обучения:", time_for_fit)

F1: 0.6887417218543046
Время обучения: 0.05309486389160156


Подберем гиперпараметры XGBoost с помощью Scikit-Learn Random Search

In [39]:
from sklearn.model_selection import RandomizedSearchCV
import warnings
import time
warnings.filterwarnings('ignore')

In [40]:
bst = XGBClassifier(random_state=121)
distributions = dict(max_depth=range(1, 21), num_estimators=range(10, 210, 10),
                     subsample=np.arange(0, 1, 0.1), reg_lambda=range(0, 100, 10), reg_alpha=range(0, 100, 10))
clf = RandomizedSearchCV(bst, distributions, random_state=0, scoring='f1')
start_time = time.time()
search = clf.fit(X, y)
search_time = time.time() - start_time
print("Подобранные гиперпараметры:", search.best_params_)
print("Полученный F1:", search.best_score_)
print("Затраченное время:", search_time)

Подобранные гиперпараметры: {'subsample': 0.30000000000000004, 'reg_lambda': 80, 'reg_alpha': 0, 'num_estimators': 190, 'max_depth': 18}
Полученный F1: 0.6498778278441029
Затраченное время: 2.5264697074890137


Теперь подберем гиперпараметры с помощью TPE и hyperopt

In [41]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

In [56]:
def objective(params):
    model = XGBClassifier(**params)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    return {'loss': -f1_score(y_test, pred), 'status': STATUS_OK}

space = {'max_depth': hp.choice('max_depth', range(1, 21)),
    'n_estimators': hp.choice('n_estimators', range(10, 210, 10)), 'subsample': hp.uniform('subsample', 0, 1),
    'reg_lamda':  hp.choice('reg_lambda', range(0, 100, 10)), "reg_alpha": hp.choice('reg_alpha', range(0, 100, 10))}

In [69]:
trials = Trials()
start_time = time.time()
best = fmin(
    fn=objective, space=space, algo=tpe.suggest, max_evals=40,
    trials=trials, show_progressbar=True)
search_time = time.time() - start_time
best_params = {'max_depth': list(range(1, 21))[best['max_depth']],
    'n_estimators': list(range(10, 210, 10))[best['n_estimators']], 'subsample': best['subsample'],
    'reg_lambda': list(range(0, 100, 10))[best['reg_lambda']], 'reg_alpha': list(range(0, 100, 10))[best['reg_alpha']]}
print("Подобранные гиперпараметры:", best_params)
print("Полученные F1:", -trials.best_trial['result']['loss'])
print("Затраченное время:", search_time)

100%|██████████| 40/40 [00:02<00:00, 19.78trial/s, best loss: -0.7051282051282051]
Подобранные гиперпараметры: {'max_depth': 3, 'n_estimators': 30, 'subsample': 0.48288156713372826, 'reg_lambda': 50, 'reg_alpha': 0}
Полученные F1: 0.7051282051282051
Затраченное время: 2.0384223461151123


Таким образом, за время меньше чем при случайном поиске мы получили F1 качество лучше, чем при ручном поиске. При этом качество при случайном поиске хуже. При случайно поиске получаем что reg_alpha стремится быть очень маленьким (0), reg_lambda 80, количество деревьев большое, глубина деревьев тоже очень большая 190. Наоборот, при поиске с помощью TPE, получаем маленькую глубину и число деревьев, reg_lambda тут тоже большой 50, а reg_alpha тоже 0.